In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import time

In [2]:
%cd /Users/11251/DSCI-525-Ci-Xu
## Change it to the location that you want to download your files to.

C:\Users\11251\DSCI-525-Ci-Xu


In [3]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'mimetype': 'image/png'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'mimetype': 'text/plain'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'mimetype': 'text/x-python'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://n

In [5]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 781 ms
Wall time: 5min 58s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 5.61 s
Wall time: 12.3 s


In [7]:
%ls -ltr figshareairline

 Volume in drive C is Windows
 Volume Serial Number is 5CA3-1071

 Directory of C:\Users\11251\DSCI-525-Ci-Xu


 Directory of C:\Users\11251\DSCI-525-Ci-Xu\figshareairline

2025/03/30  08:50    <DIR>          .
2025/03/30  08:45    <DIR>          ..
2025/03/30  08:50    <DIR>          __MACOSX
2025/03/30  08:50       127,613,760 ACCESS-CM2_daily_rainfall_NSW.csv
2025/03/30  08:50       114,707,410 ACCESS-ESM1-5_daily_rainfall_NSW.csv
2025/03/30  08:50        94,960,113 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
2025/03/30  08:50       294,260,911 BCC-CSM2-MR_daily_rainfall_NSW.csv
2025/03/30  08:50        55,224,437 BCC-ESM1_daily_rainfall_NSW.csv
2025/03/30  08:50        46,286,371 CanESM5_daily_rainfall_NSW.csv
2025/03/30  08:50       330,360,682 CMCC-CM2-HR4_daily_rainfall_NSW.csv
2025/03/30  08:50       328,787,320 CMCC-CM2-SR5_daily_rainfall_NSW.csv
2025/03/30  08:50       328,852,379 CMCC-ESM2_daily_rainfall_NSW.csv
2025/03/30  08:50       814,041,183 data.zip
2025/03/30  08:50       

File Not Found


In [8]:
# Start timing
start_time = time.time()

# Path to the folder containing all model CSV files
data_folder = "figshareairline"

# Get a list of all CSV files in the folder
all_files = glob.glob(os.path.join(data_folder, "*.csv"))

# Exclude the observed data file
csv_files = [f for f in all_files if "observed_daily_rainfall_SYD.csv" not in f]

# List to store individual dataframes
df_list = []

# Columns to read from each file
use_cols = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]

# Loop through each model file
for file in csv_files:
    # Extract the model name from the filename (text before the first underscore)
    match = re.search(r"/([^_/]+)", file.replace("\\", "/"))
    model_name = match.group(1) if match else "Unknown"

    try:
        # Read only selected columns
        df = pd.read_csv(file, usecols=use_cols)

        # Add the model name as a new column
        df["model"] = model_name

        # Append to the list
        df_list.append(df)

    except Exception as e:
        print(f"Error reading file {file}: {e}")

# Concatenate all dataframes into one
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined data to a CSV
combined_df.to_csv("combined_rainfall_models.csv", index=False)

# Print how long it took
end_time = time.time()
print(f"Data successfully combined in {end_time - start_time:.2f} seconds.")

Data successfully combined in 309.12 seconds.


In [9]:
print(df.dtypes)
print(df.shape)
print(df.head())
print(df.isnull().sum())

time              object
lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object
(3541230, 7)
                  time    lat_min   lat_max  lon_min  lon_max  rain (mm/day)  \
0  1889-01-01 12:00:00 -35.811518 -34.86911  140.625  141.875   5.727971e-17   
1  1889-01-02 12:00:00 -35.811518 -34.86911  140.625  141.875  -4.460195e-18   
2  1889-01-03 12:00:00 -35.811518 -34.86911  140.625  141.875   0.000000e+00   
3  1889-01-04 12:00:00 -35.811518 -34.86911  140.625  141.875   0.000000e+00   
4  1889-01-05 12:00:00 -35.811518 -34.86911  140.625  141.875   2.592095e-02   

     model  
0  TaiESM1  
1  TaiESM1  
2  TaiESM1  
3  TaiESM1  
4  TaiESM1  
time             0
lat_min          0
lat_max          0
lon_min          0
lon_max          0
rain (mm/day)    0
model            0
dtype: int64


In [10]:
start = time.time()

dtypes = {
    "lat_min": "float32",
    "lat_max": "float32",
    "lon_min": "float32",
    "lon_max": "float32",
    "rain (mm/day)": "float32",
    "model": "category"
}

df = pd.read_csv("combined_rainfall_models.csv", dtype=dtypes, parse_dates=["time"])

print(f"Shape: {df.shape}")
print(f"Memory usage (MB): {df.memory_usage(deep=True).sum() / 1_000_000:.2f}")

print(df["model"].value_counts())
print(df["rain (mm/day)"].describe())

end = time.time()
print(f"EDA completed in {end - start:.2f} seconds.")

df.to_parquet("combined_rainfall_models.parquet")

Shape: (62467843, 7)
Memory usage (MB): 6047.79
model
MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: count, dtype: int64
count    5.924854e+07
mean     1.901175e+00
std      5.585735e+00
min     -3.807373e-12
25%      3.838413e-06
50%      6.154947e-02
75%      1.020918e+00
max      4